In [1]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

import pandas as pd
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime, timedelta
import os

import seaborn as sns
import cmocean as cmocn

%matplotlib inline

In [2]:
fname = '/srv/data2/srai_poseidon/srai_poseidon/observation/SatelliteVsBuoy/downloads/CCMP/atBuoyLocs/CCMP_and_Buoy_09N_140W_xrr_MatchUp_120_mins_2000.nc'
testDs = xr.open_dataset(fname)

In [3]:
testDs

<xarray.Dataset>
Dimensions:          (CC_TIME: 7576, HEIGHT: 3, DEPTH: 1)
Coordinates:
  * CC_TIME          (CC_TIME) datetime64[ns] 2000-01-01 ... 2006-12-31T18:00:00
    latitude         float64 ...
    longitude        float64 ...
  * HEIGHT           (HEIGHT) float32 3.0 4.0 10.0
  * DEPTH            (DEPTH) float32 1.0
Data variables: (12/49)
    uwnd             (CC_TIME) float32 ...
    vwnd             (CC_TIME) float32 ...
    ws               (CC_TIME) float32 ...
    nobs             (CC_TIME) float32 ...
    WDIR             (CC_TIME, HEIGHT) float32 ...
    WDIR_QC          (CC_TIME, HEIGHT) float32 ...
    ...               ...
    std_AIRT         (CC_TIME) float64 ...
    mean_RELH        (CC_TIME) float64 ...
    std_RELH         (CC_TIME) float64 ...
    mean_SST - AIRT  (CC_TIME) float64 ...
    std_SST - AIRT   (CC_TIME) float64 ...
    TAO_TIME         (CC_TIME) datetime64[ns] ...
Attributes: (12/54)
    contact:                       Remote Sensing Systems, support@remss.com
    Conventions:                   CF-1.7 ACDD-1.3
    data_structure:                grid
    title:                         RSS CCMP V3.1 6-hourly surface winds (Leve...
    summary:                       RSS VAM 6-hour analyses using ERA-5 wind r...
    institute_id:                  RSS
    ...                            ...
    time_coverage_end:             2000-01-01T23:59:59Z
    platform:                      Space-based Platforms>Earth Observation Sa...
    instrument:                    Earth Remote Sensing Instruments>Passive R...
    history:                       2024-04-01T18:28:35.496946Z - netCDF gener...
    base_date:                     Y2000 M01 D01
    date_created:                  2024-04-01T18:28:35.496946Z

In [8]:
statVars =[ 'mean_WSPD', 
            'std_WSPD', 
            'mean_WSPD_10N', 
            'std_WSPD_10N', 
            'mean_U10N_x', 
            'std_U10N_x',
            'mean_U10N_y', 
            'std_U10N_y',
            'mean_cosWDIR',
            'std_cosWDIR',
            'mean_sinWDIR',
            'std_sinWDIR',
            'mean_SST',
            'std_SST',
            'mean_AIRT',
            'std_AIRT',
            'mean_RELH',
            'std_RELH',
            'mean_SST - AIRT',
            'std_SST - AIRT']

In [28]:
def getSubDF(ds, LAT, LON):
    subds = xr.Dataset()

    taoTimeVar = ['TAO_TIME']
    
    windVars = ['WDIR',
         'WDIR_QC',
         'WDIR_DM',
         'WSPD',
         'WSPD_QC',
         'WSPD_DM',
         'UWND',
         'VWND']

    sstVars = ['SST',
             'SST_QC',
             'SST_DM']
    
    airtVars = ['AIRT',
             'AIRT_QC',
             'AIRT_DM']
    
    rhVars = ['RELH',
             'RELH_QC',
             'RELH_DM']
    
    u10Vars =['WSPD_10N',
             'WSPD_10']
        
    otherVars = ['SST - AIRT']
    
    satVars = ['uwnd',
               'vwnd',
               'ws',
               'nobs'] 
    
    for windVar in windVars:
        subds[windVar] = ds[windVar].sel(HEIGHT=4)
    subds.drop_vars('HEIGHT')
    
    for sstVar in sstVars:
        subds[sstVar] = ds[sstVar].sel(DEPTH=1)
    subds.drop_vars('DEPTH')
    
    for rhVar in rhVars:
        subds[rhVar] = ds[rhVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    
    for airtVar in airtVars:
        subds[airtVar] = ds[airtVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    
    for u10Var in u10Vars:
        subds[u10Var] = ds[u10Var].sel(HEIGHT=10)
    subds.drop_vars('HEIGHT')

    for otherVar in otherVars:
        subds[otherVar] = ds[otherVar]

    for statVar in statVars:
        subds[statVar] = ds[statVar]

    subds['LATITUDE'] = xr.DataArray(LAT * np.ones(len(subds['CC_TIME']), dtype = float), dims=['CC_TIME'])
    subds['LONGITUDE'] = xr.DataArray(LON * np.ones(len(subds['CC_TIME']), dtype = float), dims=['CC_TIME'])
    
    subds['TAO_TIME'] = ds['TAO_TIME']

    deployFileName = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    ds2 = xr.open_dataset(deployFileName)
    startDates = np.array(ds2['startDate'])
    endDates = np.array(ds2['endDate'])

    depNum = np.zeros((len(subds['CC_TIME'])), dtype=int)
    c = 1
    for i in range(1,len(startDates)):
        mask = subds['CC_TIME'] >= startDates[i]
        mask *= subds['CC_TIME'] <= endDates[i]
        depNum[mask] = c
        c = c+1

    subds['Deployment index'] = xr.DataArray(depNum, dims=['CC_TIME'])
    
    
    satDS = ds[satVars]
    

    returnDS = xr.merge((subds, satDS))

    
    
    return returnDS

In [29]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)


allDS = xr.Dataset()
dataCount = 0
fileCount = 0
for task in taskList:
    lat = task[0]
    lon = task[1]

    LAT = lat
    LON = lon

    if lat < 0:
        latUnits = 'S'
    else:
        latUnits = 'N'

    if lon < 0:
        lonUnits = 'W'
    else:
        lonUnits = 'E'

    LON = (LON+360)%360
    lat=abs(lat)
    lon=abs(lon)

    
    matchFname = f'../../downloads/CCMP/atBuoyLocs/CCMP_and_Buoy_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_120_mins_2000.nc'
    
    if os.path.isfile(matchFname):
        print(f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_2000.nc')
        rds = xr.open_dataset(matchFname)
        ds = getSubDF(rds, LAT, LON)
        CC_TIME = ds['CC_TIME'].to_numpy()
        ds = ds.rename({'CC_TIME':'count'})
        dataLen = len(ds['count'])
        print(dataCount, dataCount+dataLen)
        ds['count'] = xr.DataArray(pd.Series(np.arange(dataCount, dataCount+dataLen)), 
                                   dims = ['count'])
        ds['CC_TIME'] = xr.DataArray(CC_TIME, 
                                   dims = ['count'])
        dataCount += dataLen
        if fileCount == 0:
            allDS = ds
        else:
            allDS = xr.concat((allDS, ds), dim='count')
        fileCount += 1
    # else:
    #     print(matchFname + ' NOT FOUND')


T_08S_095W_xrr_MatchUp_2000.nc
0 6214
T_08S_110W_xrr_MatchUp_2000.nc
6214 10458


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_125W_xrr_MatchUp_2000.nc
10458 15000


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_155W_xrr_MatchUp_2000.nc
15000 19561


...

/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08N_165E_xrr_MatchUp_2000.nc
269888 276305


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_09N_140W_xrr_MatchUp_2000.nc
276305 283881


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [30]:
allDS = allDS.drop_vars(['HEIGHT','DEPTH'])

In [31]:
goodDataSet = allDS.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'uwnd': 'Zonal Wind Speed (CCMP)',
                  'vwnd': 'Meridional Wind Speed (CCMP)',
                  'ws': 'Wind Speed (CCMP)'})

In [32]:
goodData = allDS.isel(count = allDS["WSPD_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["WDIR_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["SST_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["RELH_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["AIRT_QC"].isin([1,2]))
    


print(len(goodData['count']))



283881


In [39]:
mask = np.isnan(goodData['WSPD'].to_numpy())
mask = np.logical_or(mask, np.isnan(goodData['WDIR'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['ws'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['uwnd'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['vwnd'].to_numpy()))
mask = np.logical_or(goodData['WSPD'] > 1000, mask)
mask = np.logical_or(mask, goodData['WDIR'] > 3600)
mask = np.logical_or(mask, np.isnan(goodData['WSPD'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['WDIR'].to_numpy()))

for var in statVars:
    mask = np.logical_or(mask, np.isnan(goodData[var].to_numpy()))
    
print(np.sum(mask))
df = goodData.isel(count = ~mask)
#df = df.to_dataframe()

<xarray.DataArray 'WSPD' ()>
array(2382)


In [40]:
mask

<xarray.DataArray 'WSPD' (count: 283881)>
array([ True, False, False, ..., False, False, False])
Coordinates:
  * count      (count) int64 0 1 2 3 4 5 ... 283876 283877 283878 283879 283880
    latitude   (count) float64 -8.125 -8.125 -8.125 -8.125 ... 8.875 8.875 8.875
    longitude  (count) float64 264.9 264.9 264.9 264.9 ... 219.9 219.9 219.9

In [41]:
df

<xarray.Dataset>
Dimensions:           (count: 281499)
Coordinates:
  * count             (count) int64 1 2 3 4 5 ... 283877 283878 283879 283880
    latitude          (count) float64 -8.125 -8.125 -8.125 ... 8.875 8.875 8.875
    longitude         (count) float64 264.9 264.9 264.9 ... 219.9 219.9 219.9
Data variables: (12/49)
    WDIR              (count) float32 135.9 148.9 140.8 ... 211.4 216.0 202.4
    WDIR_QC           (count) float32 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0
    WDIR_DM           (count) |S1 b'D' b'D' b'D' b'D' ... b'D' b'D' b'D' b'D'
    WSPD              (count) float32 5.75 6.65 6.36 5.25 ... 10.4 9.56 12.45
    WSPD_QC           (count) float32 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0
    WSPD_DM           (count) |S1 b'D' b'D' b'D' b'D' ... b'D' b'D' b'D' b'D'
    ...                ...
    Deployment index  (count) int64 0 0 0 0 0 0 0 0 ... 10 10 10 10 10 10 10 10
    uwnd              (count) float64 -3.924 -4.766 -4.781 ... -10.8 -9.007
    vwnd              (count) float64 4.761 6.335 3.722 ... -6.964 -6.052 -7.802
    ws                (count) float64 6.17 7.928 6.059 ... 12.32 12.38 11.92
    nobs              (count) float64 3.0 1.0 1.0 1.0 1.0 ... 2.0 0.0 0.0 0.0
    CC_TIME           (count) datetime64[ns] 2000-01-01T06:00:00 ... 2006-12-...

In [42]:
df = df.rename_vars({'WSPD':'Wind Speed (TAO)',
                     'WDIR':'Wind Direction (TAO)',
                     'UWND': 'Zonal Wind Speed (TAO)',
                     'VWND': 'Meridional Wind Speed (TAO)',
                     'SST': 'Sea Surface Temperature (TAO)',
                     'RELH': 'Relative Humidity (TAO)',
                     'AIRT': 'Air Temperature (TAO)',
                     'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                     'WSPD_10': 'Wind Speed at 10m (TAO)',
                     'uwnd': 'Zonal Wind Speed (CCMP)',
                     'vwnd': 'Meridional Wind Speed (CCMP)',
                     'ws'  : 'Wind Speed (CCMP)'})

In [43]:
df['Wind Direction (CCMP)'] = np.rad2deg(np.arctan2(df['Meridional Wind Speed (CCMP)'], df['Zonal Wind Speed (CCMP)']))%360
df['Speed Difference (CCMP - TAO)'] = df['Wind Speed (CCMP)'] - df['Neutral Wind Speed at 10m (TAO)']

# ## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
### already done
# df['Wind Direction (TAO)'] = (-(df['Wind Direction (TAO)'] - 90.0) + 360)%360

## calculate zonal and meridional winds 
df['Zonal Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.cos(np.deg2rad(df['Wind Direction (TAO)']))
df['Meridional Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.sin(np.deg2rad(df['Wind Direction (TAO)']))

## direction diff in range (-180,180)
dirDiff = ((df['Wind Direction (CCMP)'] - df['Wind Direction (TAO)'])+360)%360
dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
df['Direction Difference (CCMP - TAO)'] = dirDiff

df['Zonal Wind Speed Difference (CCMP - TAO)'] = df['Zonal Wind Speed (CCMP)'] - df['Zonal Neutral Wind Speed (TAO)']
df['Meridional Wind Speed Difference (CCMP - TAO)'] = df['Meridional Wind Speed (CCMP)'] - df['Meridional Neutral Wind Speed (TAO)']

# for timeWindow in range(20,130,10):
#     df[f'Speed Difference (CCMP - TAO {timeWindow} min mean)'] = df['Wind Speed (CCMP)'] - df[f'mean WSPD_10N {timeWindow}min']

#     ## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
#     df[f'mean WDIR {timeWindow}min'] = (-(df[f'mean WDIR {timeWindow}min'] - 90.0) + 360)%360

#     ## direction diff in range (-180,180)
#     dirDiff = ((df['Wind Direction (CCMP)'] - df[f'mean WDIR {timeWindow}min'])+360)%360
#     dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
#     df[f'Direction Difference (CCMP - TAO {timeWindow} min mean)'] = dirDiff

#     ### 
#     df[f'Zonal Wind Speed Difference (CCMP - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (CCMP)'] - df[f'mean Ux_10N {timeWindow}min']
#     df[f'Meridional Wind Speed Difference (CCMP - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (CCMP)'] - df[f'mean Vy_10N {timeWindow}min']

In [44]:
df.to_netcdf('CCMPandBuoyDataBadQualityRemovedMatchup.nc')